# Initialize

## from tracks

In [ ]:
%sx mount /media/data
from trackanalysis import *
from anastore.api import __TASKS__, modifyclasses, DELETE, CNT
TRACKS = TracksDict("/media/data/sirius/Andreas/ssHP6_Sequencing_20180102/*/*.ana", match = ".*OR3_mix_(...)_.*")
TRACKS['ref'] = "/media/data/sirius/Andreas/ssHP6_Sequencing_20180102/Reference_oligo/Test_020_044_ssHP6_OR3-10_100nM_OR3-11_20nM_after_saturation.ana"
patt       = re.compile(".*_OR3_mix_(.*?)_.*")
def _remfits(data):
    def _mod(x):
        if patt.match(x[CNT][0]):
            x[CNT] = [TRACKS[patt.match(x[CNT][0]).group(1)].path[0][:-3]+"trk"]
        else:
            x[CNT] = [TRACKS['ref'].path[0][:-3]+"trk"]
        return x

    modifyclasses(data,
                  "model.task.track.TrackReaderTask",
                  {'path': _mod},
                  "peakcalling.processor.fittoreference.FitToReferenceTask",
                  DELETE,
                  "peakcalling.processor.fittohairpin.FitToHairpinTask",
                  DELETE)
    return data

__TASKS__.patch(_remfits)
TRACKS.save("/home/pol/Documents/tracks/rnasequencing/ssHP6_Sequencing_20180102/")

## from pk

In [ ]:
from trackanalysis import *

TRACKS = TracksDict("/home/pol/Documents/tracks/rnasequencing/ssHP6_Sequencing_20180102/*.pk")
TRACKS['GCA'].tasks.subtraction            = 0, 1, 4, 6, 8, 9
TRACKS['GCA'].tasks.cleaning.maxsaturation = 100.

# Viewing Data

In [ ]:
%%opts Curve(alpha=0.5)
TRACKS.cleancycles

# Alignments

In [ ]:
import sys
sys.path.append("/home/pol/trackanalysis/analyses/")
from aligningexperiments import *

SHELF = LazyShelf("/home/pol/Documents/tracks/rnasequencing/ssHP6_Sequencing_20180102/shelf")
SHELF["PEAKS"] = lambda: createpeaks([18])

In [ ]:
align = PeaksAlignment()
align.refalign.stretch = Range(1., .05, .02)
align.refalign.bias    = Range(0., .01, .002)
align.refalign.pivot   = Pivot.absolute

MASKS = { 18: dict(AGC = -1, GGC = -1, GCA = -1)
        }

(lambda i: align.display(SHELF["PEAKS"][SHELF["PEAKS"].bead == i], 'ref',masks = MASKS.get(i, None))
)(18)

In [ ]:
DATA = (lambda i: align(SHELF["PEAKS"][SHELF["PEAKS"].bead == i], 'ref', masks = MASKS.get(i, None))
       )(18)
BARR = DATA[lambda x: x.track=='ref'].peakposition.unique()[[3,4]] + [3e-3, -3e-3]
GOOD = (DATA
        [lambda x: x.peakposition > BARR[0]]
        [lambda x: x.peakposition < BARR[1]])
pd.DataFrame({sequences.Translator.reversecomplement(i).lower(): pd.Series(GOOD[lambda x: x.track == i].peakposition.unique())
              for i in set(GOOD.track.unique()) - {'ref'}}).to_csv("/tmp/data.csv")